In [150]:
module Ev3dev

import Base: read, parse, write, call

parse(::Type{ASCIIString}, s::ASCIIString) = s
parse(::Type{Vector{ASCIIString}}, s::ASCIIString) = split(s)

immutable Brick
    root_path::AbstractString
    
    Brick(root_path::AbstractString="/") = new(root_path)
end

immutable Attribute{T, Read, Write}
    name::Symbol
    stream::IOStream
end

function call{T, Read, Write}(::Type{Attribute{T, Read, Write}}, brick::Brick, relative_path::AbstractString, name)
    path = joinpath(brick.root_path, relative_path, string(name))
    isfile(path) || error("file not found: $path")
    Read && (isreadable(path) || error("read access for attribute $(path) was requested, but file is not readable"))
    Write && (iswritable(path) || error("write access for attribute $(path) was requested, but file is not writable"))
    stream = open(path, Read, Write, false, Write, false)
    Attribute{T, Read, Write}(name, stream)
end

typealias ReadOnly{T} Attribute{T, true, false}
typealias WriteOnly{T} Attribute{T, false, true}
typealias ReadWrite{T} Attribute{T, true, true}
    
call{T, X}(attr::Attribute{T, true, X}) = parse(T, chomp(strip(readline(seekstart(attr.stream)), '\0')))
function call{T, X}(attr::Attribute{T, X, true}, value::T)
    write(seekstart(attr.stream), string(value))
    flush(attr.stream)
end

abstract AttributeSet

function call{T <: AttributeSet}(::Type{T}, brick::Brick, path::AbstractString)
    attr = T()
    for name in fieldnames(T)
        field_type = fieldtype(T, name)
        setfield!(attr, name, field_type(brick, path, name))
    end
    attr
end 

type Device{AttributesType <: AttributeSet}
    attr::AttributesType
    commands::Set{ASCIIString}
    
    function Device(brick, path)
        attr = AttributesType(brick, path)
        
        commands = Set{ASCIIString}(attr.commands())
        new(attr, commands)
    end
end

type MotorAttributes <: AttributeSet
    address::ReadOnly{ASCIIString}
    commands::ReadOnly{Vector{ASCIIString}}
    driver_name::ReadOnly{ASCIIString}
    command::WriteOnly{ASCIIString}

    count_per_rot::ReadOnly{Int}
    duty_cycle::ReadOnly{Int}
    duty_cycle_sp::ReadWrite{Int}    
    speed_sp::ReadWrite{Int}
    position_sp::ReadWrite{Int}
    polarity::ReadWrite{ASCIIString}
    stop_action::ReadWrite{ASCIIString}
    stop_actions::ReadOnly{Vector{ASCIIString}}
    
    MotorAttributes() = new()
end

typealias Motor Device{MotorAttributes}

type SensorAttributes <: AttributeSet
    address::ReadOnly{ASCIIString}
    commands::ReadOnly{Vector{ASCIIString}}
    driver_name::ReadOnly{ASCIIString}
    command::WriteOnly{ASCIIString}
    
    value0::ReadOnly{Int}
    value1::ReadOnly{Int}
    value2::ReadOnly{Int}
    value3::ReadOnly{Int}
    value4::ReadOnly{Int}
    value5::ReadOnly{Int}
    value6::ReadOnly{Int}
    value7::ReadOnly{Int}
    num_values::ReadOnly{Int}
    decimals::ReadOnly{Int}
    modes::ReadOnly{Vector{ASCIIString}}
    mode::ReadWrite{ASCIIString}
    bin_data::ReadOnly{ASCIIString}
    bin_data_format::ReadOnly{ASCIIString}
    poll_ms::ReadWrite{Int}
    
    SensorAttributes() = new()
end

typealias Sensor Device{SensorAttributes}

command(dev::Device, data::AbstractString) = in(data, dev.commands) ? dev.attr.command(data) : error("command $(data) not supported by device")

function values(sensor::Device{SensorAttributes})
    vals = Vector{Float64}(sensor.attr.num_values())
    dec = sensor.attr.decimals()
    multiplier = 10.0 ^ (-dec)
    for j = 1:length(vals)
        vals[j] = multiplier * getfield(sensor.attr, symbol(:value, j-1))()
    end
    vals
end

end

import Ev3dev

In [151]:
brick = Ev3dev.Brick("/home/pi/ev3")
motor = Ev3dev.Motor(brick, "sys/class/tacho-motor/motor0")
sensor = Ev3dev.Sensor(brick, "sys/class/lego-sensor/sensor0")

Ev3dev.Device{Ev3dev.SensorAttributes}(Ev3dev.SensorAttributes(Ev3dev.Attribute{ASCIIString,true,false}(:address,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/address>)),Ev3dev.Attribute{Array{ASCIIString,1},true,false}(:commands,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/commands>)),Ev3dev.Attribute{ASCIIString,true,false}(:driver_name,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/driver_name>)),Ev3dev.Attribute{ASCIIString,false,true}(:command,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/command>)),Ev3dev.Attribute{Int32,true,false}(:value0,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value0>)),Ev3dev.Attribute{Int32,true,false}(:value1,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value1>)),Ev3dev.Attribute{Int32,true,false}(:value2,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value2>)),Ev3dev.Attribute{Int32,true,false}(:value3,IOStream(<file /home/pi/ev3/sys/class/lego-sensor/sensor0/value3>)

In [152]:
sensor.attr.value0()

562

In [153]:
Ev3dev.values(sensor)

1-element Array{Float64,1}:
 57.0

In [154]:
motor.attr.address()

"outA"

In [155]:
motor.attr.speed_sp(500)

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/speed_sp>)

In [156]:
motor.attr.command("run-forever")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)

In [157]:
Ev3dev.command(motor, "stop")

IOStream(<file /home/pi/ev3/sys/class/tacho-motor/motor0/command>)